Tesseract learning

In [1]:
from PIL import Image
import pytesseract

print(pytesseract.image_to_string(Image.open('image.png')))

NO. Item No Description Hscode P/O# S/O# Q'ty(Pcs) Unit Price(USD) Amount(USD)
2 104-10001 NeoNatalie Basic (dark) 902300 | TN200259 | S0125175 20 124.00 2480.00
10 340-00333 MamaNatalie Compl Light 902300 | TN200259 | S0125175 10 450.00 4500.00
1 340-00533 MamaN Compl (LT) Mama-U 902300 | TN200259 | SO0125175 1 705.60 705.60
1 340-11033 NeoNatalie Spare Dk 902300 | TN200259 | SO0125175 15 51.00 765.00
1 340-11133 NeoNatalie Spare Lt 902300 | TN200259 | S0125175 3 51.00 153.00
1 340-12033 NeoNatalie body (Qt2) Dk 902300 | TN200259 | SO0125175 36.60 36.60
38 360-00133 MamaBirthie (light) 902300 | TN200259 | SO0125175 38 586.00 22268.00
8 450-00033 LM Mama-U 902300 | TN200259 | SO0125175 36 153.00 5508.00




In [8]:
# print(pytesseract.image_to_data(Image.open('image.png')))

In [12]:
# print(pytesseract.image_to_boxes(Image.open('image.png')))

the coordinates of the characters are shown there

okay, pytesseract did the job right but it works better when the image is preprocessed,ie clean, high contrast image, so let's use openCV for image preprocessing

In [4]:
import cv2
import pytesseract
from PIL import Image

image_path = "image.png"
image = cv2.imread(image_path)
# print(image)

# convert to grayscale
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# threshold is applied to make the important portion of the image to be prioritized
# _, thresh_image = cv2.threshold(gray_image, 150, 255, cv2.THRESH_BINARY)

thresh_image = cv2.adaptiveThreshold(gray_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                     cv2.THRESH_BINARY, 11, 2)
                                     

# optional step, just to check how image becomes
cv2.imwrite("text_detection.png", thresh_image)

# pytesseract only works with RGB image so let's convert
pil_image = Image.fromarray(cv2.cvtColor(thresh_image, cv2.COLOR_GRAY2RGB))

text = pytesseract.image_to_string(pil_image)

# print(text)


let's save this output in a file for later use

In [71]:
with open("raw_text.txt", "w") as file:
    file.write(text)

now, we need to detect the table in the raw text, so that we can disect data as headers and ......

let's preprocess the image, now to detect the table, lines and ....

In [34]:

import numpy as np

image_path = "image.png"
image = cv2.imread(image_path)

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                               cv2.THRESH_BINARY_INV, 15, 2)

horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (30, 1))  # (Width, Height)
thresh_image = cv2.morphologyEx(thresh_image, cv2.MORPH_OPEN, horizontal_kernel, iterations=5)

# just for checking image 
cv2.imwrite("lines_detection.png", thresh_image)

True

In [63]:
thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                               cv2.THRESH_BINARY_INV, 15, 2)
# Thin the lines with erosion
kernel = np.ones((3, 3), np.uint8)

# Morphological operation to enhance horizontal lines
horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1))
thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel, iterations=8)  # Reduced iterations
cv2.imwrite("lines_detection.png", thresh)  # Check this output

True

 # this code !!!

In [ ]:
import cv2
import numpy as np

image_path = "image.png"
image = cv2.imread(image_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Adaptive thresholding
thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                               cv2.THRESH_BINARY_INV, 15, 2)

horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (60, 1))  # Narrower kernel
thresh_horizontal = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel, iterations=1)  # Fewer iterations
# cv2.imwrite("horizontal_thresh.png", thresh_horizontal)

edges_horizontal = cv2.Canny(thresh_horizontal, 100, 200, apertureSize=3)  # Higher Canny thresholds
# cv2.imwrite("edges_horizontal.png", edges_horizontal)  # Debug

lines_horizontal = cv2.HoughLinesP(edges_horizontal, 1, np.pi / 180, 
                                   threshold=400,  # Higher threshold
                                   minLineLength=350,  # Longer minimum length
                                   maxLineGap=100)  # Smaller gap tolerance


# merging two close horizontal lines
def merge_close_horizontal_lines(lines, distance_threshold=20):
    if lines is None or len(lines) == 0:
        return []
    # Extract lines and sort by y-coordinate
    lines_list = [line[0] for line in lines]
    lines_list = sorted(lines_list, key=lambda x: x[1])  # Sort by y1
    
    merged = []
    current = lines_list[0]
    for line in lines_list[1:]:
        x1, y1, x2, y2 = line
        cx1, cy1, cx2, cy2 = current
        # Check if lines are close in y and overlap or are close in x
        if abs(y1 - cy1) < distance_threshold:
            # Check if they overlap in x or have a small gap
            if (x1 <= cx2 and x2 >= cx1) or abs(x1 - cx2) < 100 + 20:
                # Merge by taking the widest x-range
                current = [min(cx1, x1), cy1, max(cx2, x2), cy2]
            else:
                merged.append(current)
                current = [x1, y1, x2, y2]
        else:
            merged.append(current)
            current = [x1, y1, x2, y2]
    merged.append(current)
    return merged

# Apply merging
merged_horizontal_lines = merge_close_horizontal_lines(lines_horizontal, distance_threshold=200)

lines_horizontal = lines_horizontal




vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 50))  # Taller kernel
thresh_vertical = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, vertical_kernel, iterations=1)  # Fewer iterations
# cv2.imwrite("vertical_thresh.png", thresh_vertical)
edges_vertical = cv2.Canny(thresh_vertical, 50, 150, apertureSize=3)
# cv2.imwrite("edges_vertical.png", edges_vertical)  # Debug edges
lines_vertical = cv2.HoughLinesP(edges_vertical, 1, np.pi / 180, threshold=100,  # Lower threshold
                                 minLineLength=200, maxLineGap=30)

# Combine and classify
horizontal_lines = []
vertical_lines = []
min_line_length = 50

# Horizontal lines from horizontal processing
if lines_horizontal is not None:
    for line in lines_horizontal:
        x1, y1, x2, y2 = line[0]
        if abs(y2 - y1) < 20 and abs(x2 - x1) > min_line_length:
            horizontal_lines.append([x1, y1, x2, y2])

# Vertical lines from vertical processing
if lines_vertical is not None:
    for line in lines_vertical:
        x1, y1, x2, y2 = line[0]
        if abs(x2 - x1) < 20 and abs(y2 - y1) > min_line_length:
            vertical_lines.append([x1, y1, x2, y2])

print(f"Horizontal lines: {len(horizontal_lines)}, Vertical lines: {len(vertical_lines)}")

# Visualize
image_with_lines = image.copy()
for line in horizontal_lines:
    x1, y1, x2, y2 = line
    cv2.line(image_with_lines, (x1, y1), (x2, y2), (0, 255, 0), 2)
for line in vertical_lines:
    x1, y1, x2, y2 = line
    cv2.line(image_with_lines, (x1, y1), (x2, y2), (0, 0, 255), 2)
cv2.imwrite("lines_detected.png", image_with_lines)

Horizontal lines: 11, Vertical lines: 10


True

# above!!!

detecting lines in table

In [66]:
# edges = cv2.Canny(thresh, 50, 150, apertureSize=3)

# # Detect lines using Hough Transform
# # lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=20, 
# #                         minLineLength=265, maxLineGap=10)


# lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=155, 
#                         minLineLength=261, maxLineGap=100)

# lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=200, 
#                         minLineLength=300, maxLineGap=20)

# # print(lines)
# image_with_lines = image.copy()

# for line in lines:
#     x1, y1, x2, y2 = line[0]
#     cv2.line(image_with_lines, (x1, y1), (x2, y2), (0, 255, 0), 2)
    

# cv2.imwrite("lines_detected.png",image_with_lines)

True

now that lines are detected, we need to separate vertical and horizotal lines

In [123]:
# Sort lines for consistent cell boundaries
horizontal_lines = sorted(horizontal_lines, key=lambda x: x[1])  # Sort by y-coordinate
vertical_lines = sorted(vertical_lines, key=lambda x: x[0])    # Sort by x-coordinate

# Detect cells
cells = []
for i in range(len(horizontal_lines) - 1):  # Iterate over rows
    for j in range(len(vertical_lines) - 1):  # Iterate over columns
        # Define cell boundaries
        x1 = vertical_lines[j][0]      # Left x-coordinate (from vertical line)
        y1 = horizontal_lines[i][1]    # Top y-coordinate (from horizontal line)
        x2 = vertical_lines[j + 1][0]  # Right x-coordinate (next vertical line)
        y2 = horizontal_lines[i + 1][1] # Bottom y-coordinate (next horizontal line)
        cells.append((x1, y1, x2, y2))

# Optional: Visualize cells on the image
image_with_cells = image.copy()
for (x1, y1, x2, y2) in cells:
    cv2.rectangle(image_with_cells, (x1, y1), (x2, y2), (0, 0, 255), 2)  # Red rectangles
cv2.imwrite("cells_detected.png", image_with_cells)

# print(f"Detected {len(cells)} cells")

True

we detected the text in the very beginning, but then why are we doing these steps again??
well, that was a random text block, but we need the the text along with the cell information
,,, in which cell the data is so that we can form the json format according to that

In [138]:
cell_text = []
for idx, (x1, y1, x2, y2) in enumerate(cells):
    # Crop the cell from the original image
    cell_image = image[y1:y2, x1:x2]
    
    # Convert to RGB for PIL (pytesseract requires RGB)
    cell_image_rgb = cv2.cvtColor(cell_image, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(cell_image_rgb)
    
    # Extract text using pytesseract
    text = pytesseract.image_to_string(pil_image).strip()
    
    # Store the text and coordinates
    cell_text.append({"text": text, "coords": (x1, y1, x2, y2)})
    # print(f"Cell {idx + 1} Text: '{text}'")

In [140]:
for i in range(9):
    print(cell_text[i])

{'text': 'NO.', 'coords': (np.int32(1), np.int32(0), np.int32(84), np.int32(26))}
{'text': 'Item No', 'coords': (np.int32(84), np.int32(0), np.int32(282), np.int32(26))}
{'text': 'Description', 'coords': (np.int32(282), np.int32(0), np.int32(950), np.int32(26))}
{'text': 'Hscode', 'coords': (np.int32(950), np.int32(0), np.int32(1070), np.int32(26))}
{'text': 'P/O#', 'coords': (np.int32(1070), np.int32(0), np.int32(1215), np.int32(26))}
{'text': 'S/O#', 'coords': (np.int32(1215), np.int32(0), np.int32(1355), np.int32(26))}
{'text': "O'ty(Pcs)", 'coords': (np.int32(1355), np.int32(0), np.int32(1505), np.int32(26))}
{'text': 'Unit Price(USD)', 'coords': (np.int32(1505), np.int32(0), np.int32(1763), np.int32(26))}
{'text': 'Amount(USD)', 'coords': (np.int32(1763), np.int32(0), np.int32(1979), np.int32(26))}


upto 9, we have our header

In [142]:
cell_text = sorted(cell_text, key=lambda x: x["coords"][1])  # Sort by y1

headers_raw = [cell["text"] for cell in cell_text[:9]]  # First 9 cells = header row
data_cells = cell_text[9:]  # Remaining cells = data rows

In [143]:
print(headers_raw)

['NO.', 'Item No', 'Description', 'Hscode', 'P/O#', 'S/O#', "O'ty(Pcs)", 'Unit Price(USD)', 'Amount(USD)']


# i will see later from here

In [145]:
# import json

# with open("cell_text.json", "w") as f:
#     json.dump(cell_text, f)

import json
import numpy as np

def default_converter(o):
    if isinstance(o, np.integer):
        return int(o)
    raise TypeError(f"Object {o} is not JSON serializable")

with open("cell_text.json", "w") as f:
    json.dump(cell_text, f, default=default_converter)

In [146]:
# Assume cell_text is already populated from the previous step
headers_raw = [cell["text"] for cell in cell_text[:9]]  # First row (8 cells)

# Process each header: lowercase, underscore-separated, remove non-alphabet
import re
headers = []
for header in headers_raw:
    # Convert to lowercase and replace non-alphabet with underscore
    processed = re.sub(r'[^a-z0-9]+', '_', header.lower()).strip('_')
    headers.append(processed if processed else "unknown")  # Avoid empty strings

print("Processed Headers:", headers)

Processed Headers: ['no', 'item_no', 'description', 'hscode', 'p_o', 's_o', 'o_ty_pcs', 'unit_price_usd', 'amount_usd']


In [149]:

data_cells = cell_text[9:]  # Exclude header row
rows = [data_cells[i:i + 9] for i in range(0, len(data_cells), 9)]

print(f"Total rows detected: {len(rows)}")
for idx, row in enumerate(rows):
    print(f"Row {idx + 1}: {[cell['text'] for cell in row]}")

Total rows detected: 9
Row 1: ['', '', '', '', '', '', '', '', '']
Row 2: ['', '104-10001', 'NeoNatalie Basic (dark)', '902300', 'TN200259', '$0125175', '20', '124.00', '2480.00']
Row 3: ['', '340-00333', 'MamaNatalie Comp! Light', '902300', 'TN200259', '$0125175', '', '450.00', '4500.00']
Row 4: ['', '340-00533', 'MamaN Compl (LT) Mama-U', '902300', 'TN200259', '$0125175', '', '705.60', '705.60']
Row 5: ['', '340-11033', 'NeoNatalie Spare Dk', '902300', 'TN200259', '$0125175', '', '', '765.00']
Row 6: ['', '340-11133', 'NeoNatalie Spare Lt', '902300', 'TN200259', '$0125175', '', '', '153.00']
Row 7: ['', '340-12033', 'NeoNatalie body (Qt2) Dk', '902300', 'TN200259', '$0125175', '', '', '36.60']
Row 8: ['', '360-00133', 'MamaBirthie (light)', '902300', 'TN200259', '$0125175', '', '586.00', '22268.00']
Row 9: ['', '450-00033', 'LM Mama-U', '902300', 'TN200259', '$0125175', '', '153.00', '5508.00']


In [150]:
# Create a list of dictionaries
json_data = []
for row in rows:
    row_dict = {}
    for header, cell in zip(headers, [cell["text"] for cell in row]):
        row_dict[header] = cell
    json_data.append(row_dict)

print("JSON Data:", json_data)

JSON Data: [{'no': '', 'item_no': '', 'description': '', 'hscode': '', 'p_o': '', 's_o': '', 'o_ty_pcs': '', 'unit_price_usd': '', 'amount_usd': ''}, {'no': '', 'item_no': '104-10001', 'description': 'NeoNatalie Basic (dark)', 'hscode': '902300', 'p_o': 'TN200259', 's_o': '$0125175', 'o_ty_pcs': '20', 'unit_price_usd': '124.00', 'amount_usd': '2480.00'}, {'no': '', 'item_no': '340-00333', 'description': 'MamaNatalie Comp! Light', 'hscode': '902300', 'p_o': 'TN200259', 's_o': '$0125175', 'o_ty_pcs': '', 'unit_price_usd': '450.00', 'amount_usd': '4500.00'}, {'no': '', 'item_no': '340-00533', 'description': 'MamaN Compl (LT) Mama-U', 'hscode': '902300', 'p_o': 'TN200259', 's_o': '$0125175', 'o_ty_pcs': '', 'unit_price_usd': '705.60', 'amount_usd': '705.60'}, {'no': '', 'item_no': '340-11033', 'description': 'NeoNatalie Spare Dk', 'hscode': '902300', 'p_o': 'TN200259', 's_o': '$0125175', 'o_ty_pcs': '', 'unit_price_usd': '', 'amount_usd': '765.00'}, {'no': '', 'item_no': '340-11133', 'desc

In [151]:
# Save to JSON file
with open("output.json", "w") as f:
    json.dump(json_data, f, indent=4)

print("JSON file saved as 'output.json'")

JSON file saved as 'output.json'


In [258]:
import zipfile

# Create a zip file
with zipfile.ZipFile("submission.zip", "w", zipfile.ZIP_DEFLATED) as zipf:
    zipf.write("output.json")
    zipf.write("image.png")

print("Submission zip file created as 'submission.zip'")

Submission zip file created as 'submission.zip'
